In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def top_league(n):

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    links = pageSoup.find_all("tr")

    site = 'https://www.transfermarkt.co.uk'
    s20_string = "/plus/?saison_id=2020"

    top_leagues = []
    for a in list(range(2,((n*2)+1),2)):
        top_leagues.append((site+(str(links[a]).split('href="',5)[2].split('"')[0])+s20_string))

    teams = []

    for a in top_leagues:

        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        links2 = pageSoup.find_all("a",{"class":"vereinprofil_tooltip"})
        site = 'https://www.transfermarkt.co.uk'
        s21_string = "/plus/?saison_id=2021"

        if a=="https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2020" or a=="https://www.transfermarkt.co.uk/liga-portugal-bwin/startseite/wettbewerb/PO1/plus/?saison_id=2020" or a=="'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=2020'":
            for b in list(range(0,(18*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
        elif a== 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1/plus/?saison_id=2020':
            for b in list(range(0,(16*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string) 
        else:
            for b in list(range(0,(20*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
    return teams

In [242]:
def top_league_player(n):

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    links = pageSoup.find_all("tr")

    site = 'https://www.transfermarkt.co.uk'
    s20_string = "/plus/?saison_id=2020"

    top_leagues = []
    for a in list(range(2,((n*2)+1),2)):
        top_leagues.append((site+(str(links[a]).split('href="',5)[2].split('"')[0])+s20_string))

    teams = []

    for a in top_leagues:

        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        links2 = pageSoup.find_all("a",{"class":"vereinprofil_tooltip"})
        site = 'https://www.transfermarkt.co.uk'
        s21_string = "/plus/?saison_id=2021"

        if a=="https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2020" or a=="https://www.transfermarkt.co.uk/liga-portugal-bwin/startseite/wettbewerb/PO1/plus/?saison_id=2020" or a=="'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=2020'":
            for b in list(range(0,(18*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
        elif a== 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1/plus/?saison_id=2020':
            for b in list(range(0,(16*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string) 
        else:
            for b in list(range(0,(20*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
    
    player_links = []
    for a in teams:
        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        site = 'https://www.transfermarkt.co.uk'

        links = pageSoup.find_all("a",{"class":"spielprofil_tooltip"})

        for b in list(range(0,len(links),2)):
            player_links.append(site+(str(links[b]).split('href="')[1].split('"')[0]))
    
    return player_links
    

In [77]:
everyone = top_league_player(5)
everyone[0:20]

NameError: name 'top_league_player' is not defined

In [244]:
len(everyone)

3320

In [55]:
england = top_league(1)
england

['https://www.transfermarkt.co.uk/fc-liverpool/startseite/verein/31/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/manchester-city/startseite/verein/281/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-chelsea/startseite/verein/631/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/tottenham-hotspur/startseite/verein/148/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-everton/startseite/verein/29/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/leicester-city/startseite/verein/1003/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/wolverhampton-wanderers/startseite/verein/543/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/west-ham-united/startseite/verein/379/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/aston-villa/startseite/ve

In [245]:
teams = top_league(5)
teams

['https://www.transfermarkt.co.uk/fc-liverpool/startseite/verein/31/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/manchester-city/startseite/verein/281/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-chelsea/startseite/verein/631/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/tottenham-hotspur/startseite/verein/148/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-everton/startseite/verein/29/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/leicester-city/startseite/verein/1003/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/wolverhampton-wanderers/startseite/verein/543/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/west-ham-united/startseite/verein/379/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/aston-villa/startseite/ve

In [246]:
len(teams)

98

In [39]:
#Creating a function that takes a list of links as an argument. 
#The function will scrape through the provided links' corresponding webpages and return a dataframe of the necessary scraped player data.
def transfermarkt(link_list):

    headers = {'User-Agent': 
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    club_links = link_list

    for a in club_links:

        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        Age = pageSoup.find_all("td", {"class": "zentriert"})
        Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
        Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
        Nationality = pageSoup.find_all("td", {"class": "zentriert"})
        Club = pageSoup.find_all("meta")
        League = pageSoup.find_all("meta")


        if a == club_links[0]:
            PlayersList = []
            AgeList=[]
            NationList=[]
            ValuesList = []
            PositionsList=[]
            ClubList = []
            LeagueList = []

            for i in range(0,len(Values)):
                ValuesList.append(Values[i].text)

            for i in range(0,len(Players)):
                PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

            for i in range(0,len(Positions)):
                PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

            for i in range(1,(len(Players)*3),3):
                AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

            for i in range(2,(len(Players)*3),3):
                NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])
                
            for i in range(0,len(Players)):
                ClubList.append(str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0])
                
            for i in range(0,len(Players)):
                LeagueList.append(League)
            

        else:

            for i in range(0,len(Values)):
                ValuesList.append(Values[i].text)

            for i in range(0,len(Players)):
                PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

            for i in range(0,len(Positions)):
                PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

            for i in range(1,(len(Players)*3),3):
                AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

            for i in range(2,(len(Players)*3),3):
                NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])
            
            for i in range(0,len(Players)):
                ClubList.append(str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0])
            
            for i in range(0,len(Players)):
                LeagueList.append(str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0])


    #Initial uncleaned dataframe initiated
    df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"League":LeagueList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList,"Values":ValuesList,})
    
    #Missing Transfer Values were stored as '\xa0'. The line below replaces them as None values
    df.loc[(df.Values == '\xa0'),'Values']= None

    #Dataframe without missing transfer values
    df_droppednull = df.dropna()

    #Converting the Transfer Values data to float            
    transfer_values = [a[1:len(a)-2] for a in df_droppednull['Values']]

    cleaned_values=[]
    for a in list(range(len(transfer_values))):
        if transfer_values[a].endswith('m'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-1])*1000000)
        elif transfer_values[a].endswith('Th.'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-3])*1000)
        else:
            cleaned_values.append

    #Constructing the Final Dataframe   
    final_df= pd.DataFrame({"Player":df_droppednull['Players'],"Club":df_droppednull['Club'],"League":df_droppednull['League'],
                            "Age":df_droppednull['Age'],"Position":df_droppednull['Position'],
                            "Nation":df_droppednull['Nationality'],"Value":cleaned_values})

    #df ranked by transfer value
    ranked_df = final_df.sort_values('Value',ascending=False)
    return ranked_df

In [40]:
transfermarkt_df = transfermarkt(england)

In [ ]:
transfermarkt_df[]

FIXING THE TRANSFERMARKT CLUB PAGE SCRAPE

In [32]:
top_3_leagues = top_league(3)

In [33]:
top_3_leagues

['https://www.transfermarkt.co.uk/fc-liverpool/startseite/verein/31/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/manchester-city/startseite/verein/281/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-chelsea/startseite/verein/631/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/tottenham-hotspur/startseite/verein/148/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/fc-everton/startseite/verein/29/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/leicester-city/startseite/verein/1003/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/wolverhampton-wanderers/startseite/verein/543/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/west-ham-united/startseite/verein/379/plus/?saison_id=2021',
 'https://www.transfermarkt.co.uk/aston-villa/startseite/ve

In [42]:
headers = {'User-Agent': 
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

club_links = top_3_leagues

for a in club_links:

    page = a
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
    Age = pageSoup.find_all("td", {"class": "zentriert"})
    Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
    Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
    Nationality = pageSoup.find_all("td", {"class": "zentriert"})
    Club = pageSoup.find_all("meta")
    League = pageSoup.find_all("meta")


    if a == club_links[0]:
        PlayersList = []
        AgeList=[]
        NationList=[]
        ValuesList = []
        PositionsList=[]
        ClubList = []
        LeagueList = []

        for i in range(0,len(Values)):
            ValuesList.append(Values[i].text)

        for i in range(0,len(Players)):
            PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

        for i in range(0,len(Positions)):
            PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

        for i in range(1,(len(Players)*3),3):
            AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

        for i in range(2,(len(Players)*3),3):
            NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])

        for i in range(0,len(Players)):
            ClubList.append(str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0])

        for i in range(0,len(Players)):
            LeagueList.append(League)


    else:

        for i in range(0,len(Values)):
            ValuesList.append(Values[i].text)

        for i in range(0,len(Players)):
            PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

        for i in range(0,len(Positions)):
            PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

        for i in range(1,(len(Players)*3),3):
            AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

        for i in range(2,(len(Players)*3),3):
            NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])

        for i in range(0,len(Players)):
            ClubList.append(str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0])

        for i in range(0,len(Players)):
            LeagueList.append(League)


#Initial uncleaned dataframe initiated
df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"League":LeagueList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList,"Values":ValuesList,})

#Missing Transfer Values were stored as '\xa0'. The line below replaces them as None values
df.loc[(df.Values == '\xa0'),'Values']= None

#Dataframe without missing transfer values
df_droppednull = df.dropna()

#Converting the Transfer Values data to float            
transfer_values = [a[1:len(a)-2] for a in df_droppednull['Values']]

cleaned_values=[]
for a in list(range(len(transfer_values))):
    if transfer_values[a].endswith('m'):
        cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-1])*1000000)
    elif transfer_values[a].endswith('Th.'):
        cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-3])*1000)
    else:
        cleaned_values.append(float(a))

#Constructing the Final Dataframe   
final_df= pd.DataFrame({"Player":df_droppednull['Players'],"Club":df_droppednull['Club'],"League":df_droppednull['League'],
                    "Age":df_droppednull['Age'],"Position":df_droppednull['Position'],
                    "Nation":df_droppednull['Nationality'],"Value":cleaned_values})

#df ranked by transfer value
ranked_df = final_df.sort_values('Value',ascending=False)

In [43]:
ranked_df

,Player,Club,League,Age,Position,Nation,Value
119,Harry Kane,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",28,attack,England,108000000.0
90,Romelu Lukaku,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",28,attack,Belgium,90000000.0
26,Mohamed Salah,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",29,attack,Egypt,90000000.0
48,Kevin De Bruyne,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",30,midfield,Belgium,90000000.0
145,Jadon Sancho,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",21,attack,England,90000000.0
...,...,...,...,...,...,...,...
1669,Petar Zovko,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",19,Goalkeeper,Bosnia-Herzegovina,68000.0
1581,Nicola Borghetto,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",21,Goalkeeper,Italy,45000.0
1582,Alessandro Berardi,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",30,Goalkeeper,Italy,45000.0
1676,Leonardo Gabelli,""" property=""og:description""/>","[[], [], [], [\n, [], \n, [], \n, [], \n, [], ...",19,Defender,Italy,23000.0


In [10]:
ClubList

['" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="og:description"/>',
 '" property="

Performing transfermarkt scrape on a single club

In [53]:
headers = {'User-Agent': 
               'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = 'https://www.transfermarkt.co.uk/manchester-city/startseite/verein/281/plus/?saison_id=2021'
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
Age = pageSoup.find_all("td", {"class": "zentriert"})
Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
Nationality = pageSoup.find_all("td", {"class": "zentriert"})
Club = pageSoup.find_all("meta")
League = pageSoup.find_all("meta")



PlayersList = []
AgeList=[]
NationList=[]
ValuesList = []
PositionsList=[]
ClubList = []
LeagueList = []

for i in range(0,len(Values)):
    ValuesList.append(Values[i].text)

for i in range(0,len(Players)):
    PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

for i in range(0,len(Positions)):
    PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

for i in range(1,(len(Players)*3),3):
    AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

for i in range(2,(len(Players)*3),3):
    NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])

for i in range(0,len(Players)):
    ClubList.append(str(Club[6]).split('content="')[1].split(",")[0])

for i in range(0,len(Players)):
    LeagueList.append(str(Club[6]).split('content="')[1].split(",")[1])       


#Initial uncleaned dataframe initiated
df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"League":LeagueList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList,"Values":ValuesList,})

#Missing Transfer Values were stored as '\xa0'. The line below replaces them as None values
df.loc[(df.Values == '\xa0'),'Values']= None

#Dataframe without missing transfer values
df_droppednull = df.dropna()

#Converting the Transfer Values data to float            
transfer_values = [a[1:len(a)-2] for a in df_droppednull['Values']]

cleaned_values=[]
for a in list(range(len(transfer_values))):
    if transfer_values[a].endswith('m'):
        cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-1])*1000000)
    elif transfer_values[a].endswith('Th.'):
        cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-3])*1000)
    else:
        cleaned_values.append(float(a))

#Constructing the Final Dataframe   
final_df= pd.DataFrame({"Player":df_droppednull['Players'],"Club":df_droppednull['Club'],"League":df_droppednull['League'],
                "Age":df_droppednull['Age'],"Position":df_droppednull['Position'],
                "Nation":df_droppednull['Nationality'],"Value":cleaned_values})

#df ranked by transfer value
ranked_df = final_df.sort_values('Value',ascending=False)

In [54]:
ranked_df

,Player,Club,League,Age,Position,Nation,Value
16,Kevin De Bruyne,Manchester City,Premier League,30,midfield,Belgium,90000000.0
18,Raheem Sterling,Manchester City,Premier League,26,attack,England,81000000.0
19,Phil Foden,Manchester City,Premier League,21,attack,England,72000000.0
3,Rúben Dias,Manchester City,Premier League,24,Defender,Portugal,67500000.0
12,Rodri,Manchester City,Premier League,25,midfield,Spain,63000000.0
17,Bernardo Silva,Manchester City,Premier League,27,midfield,Portugal,63000000.0
20,Jack Grealish,Manchester City,Premier League,25,attack,England,58500000.0
24,Gabriel Jesus,Manchester City,Premier League,24,attack,Brazil,54000000.0
10,João Cancelo,Manchester City,Premier League,27,Defender,Portugal,49500000.0
0,Ederson,Manchester City,Premier League,27,Goalkeeper,Brazil,45000000.0


Implementing the above working function into a scalable loop

In [75]:
PlayersList = []
AgeList=[]
NationList=[]
ValuesList = []
PositionsList=[]
ClubList = []


for a in top_league(5):

    headers = {'User-Agent': 
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = a
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
    Age = pageSoup.find_all("td", {"class": "zentriert"})
    Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
    Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
    Nationality = pageSoup.find_all("td", {"class": "zentriert"})
    Club = pageSoup.find_all("meta")
    


    for i in range(0,len(Values)):
        ValuesList.append(Values[i].text)

    for i in range(0,len(Players)):
        PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

    for i in range(0,len(Positions)):
        PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

    for i in range(1,(len(Players)*3),3):
        AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

    for i in range(2,(len(Players)*3),3):
        NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])

    for i in range(0,len(Players)):
        ClubList.append(str(Club[6]).split('content="')[1].split(",")[0])

      

#Initial uncleaned dataframe initiated
df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList,"Values":ValuesList,})

#Missing Transfer Values were stored as '\xa0'. The line below replaces them as None values
df.loc[(df.Values == '\xa0'),'Values']= None

#Dataframe without missing transfer values
df_droppednull = df.dropna()

#Converting the Transfer Values data to float            
transfer_values = [a[1:len(a)-2] for a in df_droppednull['Values']]

cleaned_values=[]
for a in list(range(len(transfer_values))):
    if transfer_values[a].endswith('m'):
        cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-1])*1000000)
    elif transfer_values[a].endswith('Th.'):
        cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-3])*1000)
    else:
        cleaned_values.append(float(a))

#Constructing the Final Dataframe   
final_df= pd.DataFrame({"Player":df_droppednull['Players'],"Club":df_droppednull['Club'],
                "Age":df_droppednull['Age'],"Position":df_droppednull['Position'],
                "Nation":df_droppednull['Nationality'],"Value":cleaned_values})

#df ranked by transfer value
ranked_df = final_df.sort_values('Value',ascending=False)

In [76]:
ranked_df

,Player,Club,Age,Position,Nation,Value
2255,Kylian Mbappé,Paris Saint-Germain,22,attack,France,144000000.0
1747,Erling Haaland,Borussia Dortmund,21,attack,Norway,117000000.0
119,Harry Kane,Tottenham Hotspur,28,attack,England,108000000.0
2250,Neymar,Paris Saint-Germain,29,attack,Brazil,90000000.0
26,Mohamed Salah,Liverpool FC,29,attack,Egypt,90000000.0
...,...,...,...,...,...,...
2619,Mamadou Mbow,Stade Reims,21,Defender,Senegal,45000.0
1582,Alessandro Berardi,Hellas Verona,30,Goalkeeper,Italy,45000.0
1581,Nicola Borghetto,Hellas Verona,21,Goalkeeper,Italy,45000.0
1676,Leonardo Gabelli,Spezia Calcio,19,Defender,Italy,23000.0
